In [1]:
import os
from rdkit import Chem
from rdkit import RDLogger
from model.EGCMCalculator import EGCMCalculator
import warnings
import pickle
import numpy as np
from tqdm import tqdm
warnings.filterwarnings("ignore") # hide warnings from Bio.PDB in the EGCMCalculator (TER symbol mostly)
RDLogger.DisableLog('rdApp.*') # hide warnings from RDKit (wrong valence mostly)

In [2]:
pdbbind_dir = '/home/baillifb/v2019-other-PL/'

In [3]:
pdb_ids = sorted([dir for dir in os.listdir(pdbbind_dir) if len(dir) == 4])

In [4]:
len(pdb_ids)

17652

In [5]:
pdb_ids[:5]

['10gs', '11gs', '13gs', '16pk', '184l']

In [6]:
wrong_lig_pdb_ids = []
for pdb_id in pdb_ids :
    try :
        mol = Chem.SDMolSupplier(f'{pdbbind_dir}{pdb_id}/{pdb_id}_ligand.sdf')[0]
    except :
        mol = None
        print('Impossible to read sdf file for ' + pdb_id)
        print('Trying with mol2 file')

    if mol is None :
        print('sdf file is wrong for ' + pdb_id)
        try :
            mol = Chem.rdmolfiles.MolFromMol2File(f'{pdbbind_dir}{pdb_id}/{pdb_id}_ligand.mol2')
        except :
            mol = None
            print('Impossible to read mol2 file for ' + pdb_id)

    if mol is None :
        print('Unable to parse ligand for ' + pdb_id)
        wrong_lig_pdb_ids.append(pdb_id)

sdf file is wrong for 10gs
sdf file is wrong for 11gs
sdf file is wrong for 13gs
sdf file is wrong for 1a07
sdf file is wrong for 1a08
Unable to parse ligand for 1a08
sdf file is wrong for 1a09
Unable to parse ligand for 1a09
sdf file is wrong for 1a0q
Unable to parse ligand for 1a0q
sdf file is wrong for 1a1b
Unable to parse ligand for 1a1b
sdf file is wrong for 1a1c
Unable to parse ligand for 1a1c
sdf file is wrong for 1a1e
Unable to parse ligand for 1a1e
sdf file is wrong for 1a2c
sdf file is wrong for 1a30
sdf file is wrong for 1a37
Unable to parse ligand for 1a37
sdf file is wrong for 1a3e
sdf file is wrong for 1a4g
sdf file is wrong for 1a4k
sdf file is wrong for 1a4q
sdf file is wrong for 1a4w
sdf file is wrong for 1a5g
sdf file is wrong for 1a61
sdf file is wrong for 1a7x
Unable to parse ligand for 1a7x
sdf file is wrong for 1a86
sdf file is wrong for 1a94
sdf file is wrong for 1a9m
sdf file is wrong for 1adl
sdf file is wrong for 1aht
sdf file is wrong for 1ahx
sdf file is wro

In [9]:
right_lig_pdb_ids = [pdb_id for pdb_id in pdb_ids if pdb_id not in wrong_lig_pdb_ids]

In [10]:
len(right_lig_pdb_ids)

17101

In [12]:
egcm_dir = 'data/egcms/'
egcm_calculator = EGCMCalculator()
for pdb_id in tqdm(right_lig_pdb_ids) :
    #print(pdb_id)
    mol = Chem.SDMolSupplier(f'{pdbbind_dir}{pdb_id}/{pdb_id}_ligand.sdf')[0]
    if mol is None : # sdf was wrong
        mol = Chem.rdmolfiles.MolFromMol2File(f'{pdbbind_dir}{pdb_id}/{pdb_id}_ligand.mol2')
    protein_path = f'{pdbbind_dir}{pdb_id}/{pdb_id}_protein.pdb'
    egcm = egcm_calculator.get_egcm(ligand_mol=mol, protein_path=protein_path)
    if not egcm is None :
        np.save(f'{egcm_dir}{pdb_id}_egcm.npy', egcm)

100%|██████████| 17101/17101 [5:19:11<00:00,  1.12s/it]   


In [16]:
len(os.listdir(egcm_dir))

16862